### Estrategias de Búsqueda Híbrida con Reordenamiento (Reranking)

El reordenamiento es una técnica avanzada que mejora significativamente la calidad de los resultados en sistemas RAG.

El **reordenamiento (re-ranking)** es un proceso de filtrado de segunda etapa en sistemas de recuperación, especialmente en pipelines RAG, donde:

1. **Primera etapa**: Usamos un recuperador rápido (como BM25, FAISS, o híbrido) para obtener los top-k documentos rápidamente.

2. **Segunda etapa**: Usamos un modelo más preciso pero más lento (como un cross-encoder o LLM) para re-puntuar y reordenar esos documentos según su relevancia a la consulta.

👉 **Ventaja**: Esto asegura que los documentos más relevantes aparezcan en la parte superior, mejorando significativamente la calidad de la respuesta final del LLM.

**¿Por qué es importante?**
- Los recuperadores rápidos priorizan velocidad sobre precisión
- El reordenamiento añade una capa de precisión sin sacrificar demasiado rendimiento
- Solo reordenamos un conjunto pequeño (top-k), no toda la base de datos

In [1]:
# Importación de librerías necesarias para el sistema de reordenamiento

# TextLoader: Para cargar archivos de texto plano
from langchain.document_loaders import TextLoader

# RecursiveCharacterTextSplitter: Para dividir texto en fragmentos (chunks) manejables
from langchain.text_splitter import RecursiveCharacterTextSplitter

# init_chat_model: Para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# PromptTemplate: Para crear plantillas de prompts con variables dinámicas
from langchain.prompts import PromptTemplate

# Document: Clase para representar documentos con contenido y metadatos
from langchain.schema import Document

# StrOutputParser: Para convertir la salida del LLM a string
from langchain_core.output_parsers import StrOutputParser

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Paso 1: Cargar el archivo de texto
# Cargamos un archivo de muestra que contiene información sobre LangChain
loader = TextLoader("langchain_sample.txt")
# load() devuelve una lista de documentos (en este caso, solo uno con todo el contenido)
raw_docs = loader.load()

# Paso 2: Dividir el texto en fragmentos (chunks) más pequeños
# RecursiveCharacterTextSplitter divide el texto de forma inteligente
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Tamaño máximo de cada fragmento en caracteres
    chunk_overlap=50     # Superposición entre fragmentos para mantener contexto
)
# split_documents() toma los documentos y los divide en fragmentos más pequeños
docs = splitter.split_documents(raw_docs)

# Mostrar los documentos fragmentados
docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain se integra con numerosos servicios de terceros, como OpenAI, Hugging Face y Cohere. Esto permite a los desarrolladores experimentar con diferentes modelos y optimizar el rendimiento para casos de uso especÃ\xadficos, como la sÃ\xadntesis, la respuesta a preguntas o la traducciÃ³n.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='La GeneraciÃ³n Aumentada por RecuperaciÃ³n (RAG) es una potente tÃ©cnica que recupera conocimiento externo y lo transmite a la indicaciÃ³n para fundamentar las respuestas de los LLM. LangChain facilita la i

In [4]:
# Definir la consulta del usuario
# Esta es la pregunta que queremos responder usando nuestro sistema RAG con reordenamiento
query = "¿Cómo puedo usar LangChain para construir una aplicación con memoria y herramientas?"

In [5]:
# Paso 3: Configurar FAISS con embeddings de HuggingFace

# Importar las librerías necesarias para el almacén vectorial
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Inicializar el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo compacto y eficiente (384 dimensiones)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Crear el almacén vectorial FAISS a partir de los documentos fragmentados
# FAISS convierte cada documento en un vector numérico para búsqueda de similitud
vectorstore = FAISS.from_documents(docs, embedding_model)

# Convertir el almacén en un recuperador configurado para devolver los top-8 resultados
# k=8 significa que recuperaremos los 8 documentos más similares a la consulta
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

In [6]:
# Alternativa: Configurar FAISS con embeddings de OpenAI

# Importar módulos necesarios para variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Establecer la API key de OpenAI desde las variables de entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Importar la clase de embeddings de OpenAI
from langchain_openai import OpenAIEmbeddings

# Inicializar los embeddings de OpenAI
# OpenAI usa modelos más potentes pero requiere API key y tiene costo
embeddings = OpenAIEmbeddings()

# Crear un almacén vectorial alternativo usando embeddings de OpenAI
# Esto permite comparar el rendimiento entre HuggingFace y OpenAI
vectorstore_openai = FAISS.from_documents(docs, embeddings)

# Crear recuperador con OpenAI embeddings, también configurado para top-8
retriever_openai = vectorstore_openai.as_retriever(search_kwargs={"k": 8})

In [7]:
# Mostrar la configuración del recuperador con HuggingFace embeddings
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C53FAD8CB0>, search_kwargs={'k': 8})

In [8]:
# Mostrar la configuración del recuperador con OpenAI embeddings
retriever_openai

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C541CB9DF0>, search_kwargs={'k': 8})

In [10]:
# Paso 4: Configurar el modelo de lenguaje para reordenamiento

# Importar la función para inicializar modelos de chat
from langchain.chat_models import init_chat_model

# Cargar la API key de Groq desde las variables de entorno
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Inicializar el LLM usando Groq con el modelo Gemma2-9B
# Groq ofrece inferencia rápida y este modelo es eficiente para tareas de ranking
llm = init_chat_model("groq:llama-3.1-8b-instant")

# Mostrar la configuración del LLM
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C5432F5AF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C5436C5010>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
# Paso 5: Crear el prompt para reordenamiento
# Este prompt instruye al LLM a evaluar y clasificar documentos por relevancia

prompt = PromptTemplate.from_template("""
Eres un asistente útil. Tu tarea es clasificar los siguientes documentos de más a menos relevante según la pregunta del usuario.

Pregunta del Usuario: "{question}"

Documentos:
{documents}

Instrucciones:
- Piensa cuidadosamente sobre la relevancia de cada documento respecto a la pregunta del usuario.
- Devuelve una lista de índices de documentos en orden de clasificación, comenzando por el más relevante.
- Considera tanto la coincidencia temática como la utilidad práctica para responder la pregunta.

Formato de salida: índices de documentos separados por comas (ej: 2,1,3,0,...)
""")

In [13]:
# Paso 6: Recuperar documentos usando el recuperador vectorial
# Invocamos el recuperador con nuestra consulta para obtener los documentos más similares
retrieved_docs = retriever.invoke(query)

# Mostrar los documentos recuperados antes del reordenamiento
retrieved_docs

[Document(id='a0c1f7e3-2d65-4691-9640-b7ba176e4efe', metadata={'source': 'langchain_sample.txt'}, page_content='La memoria en LangChain permite la retenciÃ³n de contexto en varios pasos de una conversaciÃ³n o tarea, lo que hace que la aplicaciÃ³n sea mÃ¡s coherente y con estado.'),
 Document(id='44be4b59-d863-4940-a9e0-4a7f82ddb285', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.'),
 Document(id='0c9c6910-5387-4e35-b2d7-47f357e7f615', metadata={'source': 'langchain_sample.txt'}, page_content='Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsque

In [15]:
# Paso 7: Crear la cadena de reordenamiento
# Esta cadena combina el prompt, el LLM y el parser de salida
# El operador | (pipe) encadena los componentes secuencialmente
chain = prompt | llm | StrOutputParser()

# Mostrar la estructura de la cadena
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nEres un asistente útil. Tu tarea es clasificar los siguientes documentos de más a menos relevante según la pregunta del usuario.\n\nPregunta del Usuario: "{question}"\n\nDocumentos:\n{documents}\n\nInstrucciones:\n- Piensa cuidadosamente sobre la relevancia de cada documento respecto a la pregunta del usuario.\n- Devuelve una lista de índices de documentos en orden de clasificación, comenzando por el más relevante.\n- Considera tanto la coincidencia temática como la utilidad práctica para responder la pregunta.\n\nFormato de salida: índices de documentos separados por comas (ej: 2,1,3,0,...)\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C5432F5AF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C5436C5010>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutput

In [16]:
# Paso 8: Formatear los documentos para el prompt
# Creamos una lista numerada de documentos con su contenido
# enumerate() nos da el índice (i) y el documento (doc) para cada elemento
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]

# Unir todos los documentos en un solo string, separados por saltos de línea
# Esto crea un formato legible para que el LLM evalúe todos los documentos
formatted_docs = "\n".join(doc_lines)

In [17]:
# Mostrar la lista de documentos formateados antes de unirlos
doc_lines

['1. La memoria en LangChain permite la retenciÃ³n de contexto en varios pasos de una conversaciÃ³n o tarea, lo que hace que la aplicaciÃ³n sea mÃ¡s coherente y con estado.',
 '2. LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.',
 '3. Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsqueda y ejecuciÃ³n de cÃ³digo.\nLangChain admite la integraciÃ³n de herramientas, como bÃºsqueda web, calculadoras y API, lo que permite que los LLM interactÃºen con sistemas externos y respondan con mayor precisiÃ³n a consultas dinÃ¡micas.',
 '4. BM25 es un mÃ©todo tradicional de recuperaciÃ³n dispersa que puntÃºa los docum

In [18]:
# Mostrar todos los documentos concatenados en un solo string
# Este es el formato que se enviará al LLM para su evaluación
formatted_docs

'1. La memoria en LangChain permite la retenciÃ³n de contexto en varios pasos de una conversaciÃ³n o tarea, lo que hace que la aplicaciÃ³n sea mÃ¡s coherente y con estado.\n2. LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.\n3. Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsqueda y ejecuciÃ³n de cÃ³digo.\nLangChain admite la integraciÃ³n de herramientas, como bÃºsqueda web, calculadoras y API, lo que permite que los LLM interactÃºen con sistemas externos y respondan con mayor precisiÃ³n a consultas dinÃ¡micas.\n4. BM25 es un mÃ©todo tradicional de recuperaciÃ³n dispersa que puntÃºa los documentos basÃ

In [19]:
# Paso 9: Invocar la cadena de reordenamiento
# Enviamos la pregunta y los documentos formateados al LLM
# El LLM analizará cada documento y devolverá los índices ordenados por relevancia
response = chain.invoke({"question": query, "documents": formatted_docs})

# Mostrar la respuesta del LLM con el ranking de documentos
response

'Después de analizar los documentos y considerar tanto la coincidencia temática como la utilidad práctica para responder la pregunta, clasifico los documentos de la siguiente manera:\n\nLa memoria en LangChain permite la retenciÃ³n de contexto en varios pasos de una conversaciÃ³n o tarea, lo que hace que la aplicaciÃ³n sea mÃ¡s coherente y con estado. (Documento 1)\nLangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes. (Documento 2)\nLos agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsqueda y ejecuciÃ³n de cÃ³digo. (Documento 3)\nLangChain admite la integraciÃ³n de herramientas, como bÃºsqueda web, calculadoras 

In [20]:
# Paso 10: Parsear la respuesta del LLM para extraer los índices
# El LLM devuelve algo como "2,1,4,5,6..." y necesitamos convertirlo a índices numéricos

# Dividimos la respuesta por comas, limpiamos espacios y filtramos solo números
# Restamos 1 porque el LLM usa numeración 1-based pero Python usa 0-based
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]

# Mostrar los índices extraídos (en formato 0-based de Python)
indices

[1, 2, 6, 4, 5, 3, 4, 1, 2, 6, 4, 5, 3]

In [21]:
# Mostrar nuevamente los documentos originales recuperados (antes del reordenamiento)
# Esto nos permite comparar el orden original vs el orden reordenado
retrieved_docs

[Document(id='a0c1f7e3-2d65-4691-9640-b7ba176e4efe', metadata={'source': 'langchain_sample.txt'}, page_content='La memoria en LangChain permite la retenciÃ³n de contexto en varios pasos de una conversaciÃ³n o tarea, lo que hace que la aplicaciÃ³n sea mÃ¡s coherente y con estado.'),
 Document(id='44be4b59-d863-4940-a9e0-4a7f82ddb285', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.'),
 Document(id='0c9c6910-5387-4e35-b2d7-47f357e7f615', metadata={'source': 'langchain_sample.txt'}, page_content='Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsque

In [22]:
# Paso 11: Reordenar los documentos según el ranking del LLM
# Usamos los índices proporcionados por el LLM para reorganizar los documentos

# List comprehension que:
# 1. Toma cada índice de la lista de indices
# 2. Verifica que el índice sea válido (0 <= i < len(retrieved_docs))
# 3. Obtiene el documento correspondiente de retrieved_docs
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]

# Mostrar los documentos reordenados
# Ahora están ordenados de más a menos relevante según el análisis del LLM
reranked_docs

[Document(id='44be4b59-d863-4940-a9e0-4a7f82ddb285', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.'),
 Document(id='0c9c6910-5387-4e35-b2d7-47f357e7f615', metadata={'source': 'langchain_sample.txt'}, page_content='Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsqueda y ejecuciÃ³n de cÃ³digo.\nLangChain admite la integraciÃ³n de herramientas, como bÃºsqueda web, calculadoras y API, lo que permite que los LLM interactÃºen con sistemas externos y respondan con mayor precisiÃ³n a consultas dinÃ¡micas.'),
 Document(id='1f3ce853-fafa-4360-b449-0c05

In [23]:
# Paso 12: Mostrar los resultados finales reordenados
# Presentamos los documentos en su nuevo orden de relevancia

print("\n📊 Resultados Finales Reordenados:")
# enumerate(reranked_docs, 1) comienza la numeración en 1 para mejor legibilidad
for i, doc in enumerate(reranked_docs, 1):
    # Mostramos cada documento con su nueva posición en el ranking
    print(f"\nRango {i}:\n{doc.page_content}")


📊 Resultados Finales Reordenados:

Rango 1:
LangChain es un framework flexible diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Proporciona herramientas y abstracciones para trabajar con LLM de forma mÃ¡s eficaz e incluye componentes para la gestiÃ³n de indicaciones, cadenas, memoria y agentes.

Rango 2:
Los agentes en LangChain son cadenas que utilizan LLM para decidir quÃ© herramientas usar y en quÃ© orden. Esto los hace adecuados para tareas de varios pasos, como la respuesta a preguntas con bÃºsqueda y ejecuciÃ³n de cÃ³digo.
LangChain admite la integraciÃ³n de herramientas, como bÃºsqueda web, calculadoras y API, lo que permite que los LLM interactÃºen con sistemas externos y respondan con mayor precisiÃ³n a consultas dinÃ¡micas.

Rango 3:
LangChain se integra con numerosos servicios de terceros, como OpenAI, Hugging Face y Cohere. Esto permite a los desarrolladores experimentar con diferentes modelos y optimizar el rendimiento para casos de us